In [ ]:
import threading
import time
import emoji
import re
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from translate import Translator #python翻譯套件
import googletrans #python google翻譯套件
from pprint import pprint
import random,time
from random import randint
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import jieba
import jieba.posseg as pseg
x=pd.read_csv('Recipe.csv',encoding='utf-8')
print(len(x['Ingredients']))
test=(x['Ingredients'])
#清除資料
def preprocess (x):
    x = emoji.get_emoji_regexp().sub(r'', x)#去貼圖
    x = re.sub(r'[（＜<《〈［(（【{][^>）＞】)}）》〉］]*[>》＞】）)}）〉］]', '', x)
    x = x.strip().replace(" ", "")
    x = re.sub(r'[~\-。]', "", x) 
   
    x=re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",x)
    x= re.sub(r'(一(包|碗|小碗|附)|(適|少)(量|當|許)|新鮮|市售|\
        |隔夜|切(段|絲|成|塊|片|碎末|花|碎|小丁|丁|末|細|條|小(塊|段))|約|(或|可(不用|用|依|選|均勻|不加|視家裡|\
        |替換|省略|切分|以)).*)|(皆|亦|即|也|均)可|(此次|一(個|支|大匙|小(撮|匙)|隻|把)|(依|視)個人|又(稱|名)).*|\
        |兩(?!節|層).*|數.*|少|個人包|成分|手切|去(皮|骨)|帶皮|現炒|原味|吃剩的|一(?!即|般)|小$.*|斜段|大碗|\
        |份量|沾醬|方便製作的|宜口為主|建議|表面裝飾|塑膠袋|喜歡|室溫|焦糖部分|焦糖材料|有機|.*刮刀|.*刀器|刀器|\
        |廚具刀|沾刀|刮刀|.*剪刀|美工刀|鋁罐|膠帶|砧板|玻璃密封罐|(or|OR)|的|加入|全聯|碗公|畫刀|料理機|食物保鮮袋|\
        |砧板|整粒|半顆|圓刀.*|抹刀|.*盤子|削皮器|溫度計|.*打蛋器|雕刻刀|美國棒球|.*蠟燭|.*水果刀|烤盤.*|.*烤模核金|.*拌盆|\
        |(大|中|小)塊|材料|中小型|冷藏|普通|退冰|完成|調味料|我用|是|蒸熟|不要太大|吋|金品|家樂福嚴選|(小|中|大)包裝|比較|\
        |煮開|(大|中|小)顆|超市可買到|(韓|日|美|台|泰)式|(日本|韓國|台灣|美國|泰國)|食材|子和風|碗|至要|http.*|北海道|罐裝|\
        |罐頭|玻璃罐|義式|炸粉|莫札|衛生|皮屑|過濾|開|帕瑪森|天使|調味|低脂|塗抹|羽衣|清|黑豚|天使|食|牙籤|桔醬|虎牌|泡發\
        |家樂福|燙熟|工研|吃剩下|泡軟|條|純正|模型|黑白|香煎|冷凍|炒香|熱騰騰|陳年|至要|含皮拍裂|古法炭焙|苦甜|小號|融化|白色|\
        |自製流淚|即用|不锈鋼.*|圓形|義美|軟化後|雀巢水滴|無鋁|現榨|扣掉內餡|烘培用|內餡|防潮|大波露|耐高溫|無糖|法式|蜜汁|速發|\
        |打發|冷|元本山|自磨|七彩|自製|汆燙|地捫|中藥店|焗烤|燙|醃漬小|半包|炸|香辣|墨西哥|燒|薄片|好菇道|碎|.*脫水器|時令|西班牙|\
        |粗粒|調漿|紅黃色|幹洗淨|淡|包去粗梗|台東|梅取|牛頭牌|隔餐掉|與|粉白|澳洲|藍象|.*造型|片丁|煙燻|其他|史雲生|烹大師|半根|綜合|\
        |水煮|隔餐|冷掉的|掉|片盒|東石|香料共和國|去骨|.*廚具刀|鄉菇香黑|普格傳統|.*鉛筆機|.*量杯|.*餐夾|.*保鮮盒|印尼|.*調理機|急凍|\
        |初榨|烘焙用|加拿大|頂級|台糖|鄉菇香黑|細|黑早|.*濾網|粗黑|.*烤箱|乾燥|芝司樂|高鈣|安心|聖女|亨氏|.*棉巾|帶肥|可改做|Prego|\
        |福壽|濃縮液|特級|李錦記|農家瑞初榨|小支|大支|東寶|雲嶺鮮雞|利用|另一半|白綠|大成鹿野土雞|築地一番鮮|手剝|新食派|中型|手工|法國|\
        |樂事|快速|綠色|.*方形模|裝飾|高融點|冰|.*烤模|常溫|天然|過篩|光泉|天然雲尼拿|浸著|湯匙|任何口味|溶化|調色|脫殼|切粒|三花|.*桿麵棍|\
        |.*烤盤|.*創意餐廚|英國|.*刮板|光合農場|林復振|組合包|得意的一天|大創DIY|用的白醬|桂冠輕鬆生活|等等|顆|量|大匙|適量|龜甲萬料亭|\
        |決定|要不要|甜度|看|娘惹|即煮醬素|等|鹹度|濃稠度|調|用|輕|竹籤|即|平葉佳|口味|吃不慣|使用鹽|請以|好侍佛蒙特|屋|圓長型|高速公路|\
        |尼諾|煮至|彈牙|不建議|混雜|口味|以免|加上|本來|有|和|超市販售|之|煮好|放涼|經典|香濃白醬|禾法頌普羅旺斯|已|煮熟|並|壓成|泥狀|\
        |小磨坊|熱炒系列|統一|便利超商|另備些|外層|及|類|使用的|圖中|不加|我|要|做成|所以|正統派士|加|極品|有心|肉舖子|月球|太平洋品|\
        |養生|純醱酵|施巴護潔露.*|廣吉|已經|發酵|變酸|莊爸|康寶|自己|熬|萬歲牌|Panasonic.*|味全|健康|廚房|黃金|龜甲萬|如果|遠東生技|\
        |纖吃|纖盈|低卡|如果|要|孩子|吃時|以|省略|前天|鹿窯菇事|料理菇事系列|無印良品|極道|風味|天然風味|料理素|水量|依泡米|長短|做|\
        |增減|.*包装|新東陽|德式|紐奧良|磨成|又|分|之|炒熟|半|滋味康|御茶|釀|匙|TaCha|廣吉|炒|鋁箔紙.*|本身|若|就|酸味|則|在|滾水中|\
        |川燙|撈起|瀝乾|夏季版|(冬|夏|秋|冬)季|大雷神|擠花嘴.*|擠花袋|公克|施巴護潔露.*|自己|動手做有時|品牌|不一樣|一樣|會|問題|濃稠度|\
        |都可|都|擠花袋.*|跟|大號|寬口徑|口徑|擠花|直徑|紙模|公分|下圍|明治|底|筷子|鋁箔紙|成棒狀|抹油|底部|先|鋪上|烘培紙|\
        |coffeegranulesdissolved.*|請|參考|免揉|大圓|任何|.*方形烤皿|剪碎|視|材料|方便性|個人|的|口感|調整|配料|普格|.種|禾法頌西南|你|喜愛|\
        |不易|出水|都|隨意|可放|跟|現成|雙活菌|烘焙|禮盒|鹹酥口味|濃縮|但|一般|那種|先用|抓過|後|醃一下|禾法頌西南|禾法頌蓋|朗德|配料|香腸世家|\
        |至|中藥行|直接|買|MoMo購物台|MoMo|.*料理紙|配菜|這個|南部|好像|只有|IKEA|的|到|研磨|香料獵人|一般|同|不需|地中海|煮過|東海岸|深海|野生|國王|屏東|雙認證|無藥|養殖|杯直接|買|現成|但|這裡|沒|在|\
        |賣|今次|選擇|去殻|六源味|永佳|上品|組|用了|先用|稍微|醃入味後|醃入|再|放入|先|抓麻|拉差|匈牙利|耗油|紅木桶|內|約|國內|超市|小盒|包裝|盒|\
        |將|.*攪拌器|打碎|橫切|橫切面|圓徑|大小|相約|李派林汁|第戎|湯瑪仕|肉舖|紐西蘭|配料|法國|諾曼第|水中|浸後|在|沸至|軟身|至|傳統|泡過|裡的|\
        |基本款|其他|冰箱|裡有|也|可以|切一切|一起|加入|以上|柯西嘉島|普羅旺斯|法蘭西島|法國|百味|來特|拿坡里|來|煮|製法|同|午茶夫人|任何|紅色|\
        |立頓|柯西嘉島|再來|五分鐘|盒|都可|澱粉類|橘色|樓氏|甲午年|五運六氣|司天之氣|塊|小方|拆絲熟|刷洗|乾淨|帶皮|打成泥|適量|白瓶|纖維|零卡|\
        |小鐵罐|.*禮物大會|卡斯特諾達禮|甜塔皮桿平|模壓|好|形狀|叉子|戳好|洞洞|迷客夏|使用|浸過|青原食譜|因為|改用|浸過|酒的|解凍後|沖洗|乾淨|\
        |茶匙|醃|分鐘|在|小磨|食在養生|杯比尤特|香腸世家|萬字|全部|混合|備用|新鲜|中種|瓶蓋|針筒|外買咖啡匙|尖刀|四個|啫喱杯|一枝|牙籤|将|全部|\
        |腌料|条状|一起|充份|拌匀|腌制|钟|解凍|鍋|中火|慢慢煮滚|滚後出|分鍾|然後|洗干凈|瀝水|备用|任何|想|增加|味的粉|無味色|香港|間中|稱為|醃料|\
        |部分|最面層|去核|切半|如|小的|多放|幾粒|先用|个|小匙|粒|适量|打结|约|数片|個|只|朵|杯|花袋|煎|汤匙|粘|拌匀|后|这是|比例|不够|将|全部|\
        |顺时钟|搅拌|均匀|成一肉馅|待用|汁料|煮成|鐵塔牌|CREUSET.*|.*鍋|去衣|最好|之前|不同|各牌子|牌子|綠山|農場|諾曼地|希臘|德國|Bertolli百得利|\
        |BERTOLLI|澎湖|海味|初|搾|西南油封|.*隨行理棒|水滴狀|卡魯哇香甜|手指|大根|準備|汁也擠出|籤|溶液|一日|玩梅|主義|帶殼|帶尾|好市多|大成|\
        |分饗|熟成|等級|現磨|五色|帶殼|帶尾|桂冠|喜馬拉雅山|麥克米奇|師父|部位|油漬|濾可|削成絲|原住民|特色|公雞牌|好市多|抹|少许|現刨|博客|威尼斯|\
        |非基因|改造|球狀|蝴蝶結|抑|家己|佮意|舒肥|邱家|兄弟|美味|精選|迷你|瑰寶|去籽|球狀|元氣|一番|好醇粹|營養|均衡|愛心|美麗媞妮|蓋世|達人|無毒|且|\
        |表皮|無蠟|第二次|浸泡|削皮|去籽|鷹仔牌|純白|屏科大|豆油伯|缸底|削皮|斜切|老乾媽|脆油|郫縣|紅油|黑咪瑪魔辣|萬家香|大吟釀|朝天|金啤|玻璃|瓶裝|九陽|\
        |智慧|溫控|紫砂煲|.*砂煲|大吟|馬玉山|液態|蝴蝶結|馬來西亞|約翰|走路|粗獷|紅牌|馬撒拉|附點|可果美|公雞牌|檳城|其餘|六|對|穀盛|有料|生活|老公|仔標|\
        |百胜廚|新加坡|益壽多|下北澤湯|前晚|剩的|多一點|當然|比較|好吃|呀|嬰兒|拳頭|大的|嘟嘟|好蒜味|安永|鮮物|伯爵牌|西西里|未漂白|現磨|雀巢|華師父|黃色|\
        |部份|百德利|慢烤|百得利|純|之前|煮|食譜|醃料|越南|工具.*|早餐|泰国|美人鱼|放軟|切小|超越|鬆脆|底部|方形|上海|2支|分割|几划|也|比较|入味|冠益|老干媽|\
        |五豐|大條|大舊|平湯匙|主要|食材|棵|小時|製作|大约|小时|涂在|大澳|秘製|打成醬|烘烤|過的|梅狀元|非常|熟|熟透|得意|又有|好味自|創意|.*廚紙巾|',"", x)
    x=re.sub("忌廉","鮮奶油",x)
    checkfood="火鍋"
    check_string=re.search(checkfood,x)
    if check_string!=None:
        #print("這是火鍋片類")
        x= re.sub('飯{2}',"飯",x)
        x=re.sub(r'[0-9]','',x)
        x=re.sub(r'／','',x)
        return x
    else:    
        x= re.sub(r'各$|各(式|種)|片狀|特級冷壓初榨|剝殼|方便製作的份量|不同|小(丁|束)|酌量|茄子重量的|沒餡|付煮熟|\
            |幾滴|裝飾用|煮切|去殼|醃料分成|鍋子.*|.*鍋|',"",x)
        x= re.sub('飯{2}',"飯",x)
        x=re.sub(r'[0-9]','',x)
        x=re.sub(r'／','',x)

        return x
#斷字斷詞中研院
def ckip_cut(cut_string):
    #GPU跑
    #os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    ws = WS("./data", disable_cuda=False)#(WS) 斷詞
    pos = POS("./data", disable_cuda=False)# (POS) 詞性標注
    ner = NER("./data", disable_cuda=False)# (NER) 實體辨識
    word_to_weight = {"玫瑰鹽": 1,"牛背脊肉": 1,}
    dictionary = construct_dictionary(word_to_weight)
    #print(dictionary)
     
    sentence_list =cut_string#cut_string要用list包住

    word_sentence_list = ws(
        sentence_list,
      
    )
    word_sentence_list = ws(sentence_list, recommend_dictionary=dictionary)
    word_sentence_list = ws(sentence_list, coerce_dictionary=dictionary)
    pos_sentence_list = pos(word_sentence_list)

    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

    del ws
    del pos
    del ner
    def print_word_pos_sentence(word_sentence, pos_sentence):
        save_word=[]
        assert len(word_sentence) == len(pos_sentence)

        for word, pos in zip(word_sentence, pos_sentence):
            #print(f"{word}({pos})", end="\u3000")
            a=word
            b=pos#顯示磁性
            #print("這個字{}的詞性是{}".format(a,b))

            if b =="Na":
                save_word.append(a)#存入暫存氣
        print(save_word)
        return save_word
     
    for i, sentence in enumerate(sentence_list):
        #print()
        #print(f"'{sentence}'")
        finsh=print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
        return finsh
    
def jieba_cut2(cut_word):
    #jieba.set_dictionary('./dict.txt')
    jieba.load_userdict('./user.txt')
    cut_list=cut_word
    finsh=[]
    for str1 in cut_list:
        seg_list = jieba.lcut(str1, cut_all=False)
        #print(seg_list)
        for i in seg_list:
            finsh.append(i)
        
        #print(a)
        #print(finsh)
        
        #print(finsh)
    
    print(finsh)
    return finsh
    
def jieba_cut(cut_word):
    #jieba.set_dictionary('./dict.txt')
    finsh=[]
    text = cut_word
    #print(text)
    words = pseg.cut(text)
    #jieba.enable_paddle()
    #words = pseg.cut(text,use_paddle=True)
    for word, flag in words:
        if flag =="n" or flag== "nr" or flag=='x' or flag=='nz':
            print(word, flag)
            finsh.append(word)
        else:
            print(word, flag)
            #pass
    print(finsh)
    return finsh 

#英轉中    
def check_english_goole(string):
    checkstring=string 
    #time.sleep(1)
    pattern='[a-zA-Z]{3,}' #英文3個字以上篩選
    pattern2='[\u4e00-\u9fa5]+'#選取中文
    #pattern2='[^[a-zA-Z]{3,}]'
    check_eng=re.search(pattern,checkstring)#判斷是否是英文單字
    check_tw=re.search(pattern2,checkstring)
    check_eng_finditer=list(re.finditer(pattern,checkstring))#用finditer找英文match值 回傳多筆資料
    check_tw_finditer=list(re.finditer(pattern2,checkstring))#用finditer找中文match值 回傳多筆資料
    check_zn_tw=re.findall(pattern2,checkstring)#中文字的範圍是個list
    #check_zn_tw= [i for i in check_zn_tw if i != '']#刪除空值
    condition=re.findall(pattern,checkstring)#英文字的範圍是個list

    set_order={}
    get_tw_order={}
    get_en_order={}
    
    for i in check_tw_finditer:
        get_order=i.start()#抓match初始位置
        get_items=i.group(0)#抓match的文字轉成str
         #print(get_items)
       
        tmp_dict={get_items:get_order}
        #print(tmp_dict)
        get_tw_order.update(tmp_dict)
        tmp_dict={}
   # print( get_tw_order)
    for j in check_eng_finditer:
        get_order=j.start()
        get_items=j.group(0)
        #print(get_items)
        #time.sleep(2)
        translator = googletrans.Translator()#用google套件
        #print("進入翻譯前")
        change_eng=translator.translate( get_items, dest='zh-tw').text#英文轉中文
        #print("進入翻譯後")
        #print(change_eng)
        #tmp_string="({},{})".format(  change_eng,get_order)
        tmp_dict={change_eng:get_order}
        #print(tmp_dict)
        get_en_order.update(tmp_dict)
        tmp_dict={}
    #print(get_en_order)
    #print(get_tw_order)
    #print(get_en_order)
    set_order.update(get_tw_order)
    set_order.update(get_en_order)
    #print(set_order)
    finsh_list=sorted(set_order.items(), key=lambda x:x[1])
    #print(finsh_list)
    finish=""
    for i in finsh_list:
        finish=finish+i[0]
    return  finish
    #重制所有dict
    set_order={}
    get_tw_order={}
    get_en_order={}
    finsh_list=[]
    
    
dict2={}
count=0
list_col=[]
for food_list in test:
    
#print(type(food_list))

    food_list=food_list.replace("、"," ")
    food_list=food_list.replace(";"," ")
    #print(food_list)
    list3=food_list.split()
    re_a_count=1
    #print(list3)
    #print(type(list2))
    #count+=1
    #count=count+1
    #print("這是第{}個欄位".format(count))
    #print("__________________________")
    save_back=''
    save_back_list=[]
    #print(count)

    #print("第{}個欄位".format(count))
    count+=1
    for a in list3:
        #print(a)
        #print(a)
        re_a=preprocess(a)

        re_a=check_english_goole(re_a)

        #re_a=preprocess(a)



        if len(re_a)>6:
            print(a)
            print("斷詞前",re_a)
            #print("字數大於0個進入斷字斷詞")
            print("第{}個欄位".format(count))
            re_a=[re_a]
            #re_a_list=ckip_cut(re_a)
            #re_a_list=jieba_cut(re_a)
            re_a_list=jieba_cut2(re_a)
            #print(re_a_list)

            for i in re_a_list:
                re_a=i

                save_back=save_back+re_a+" "

                if re_a in dict2:

                    dict2[re_a]+=1

                else:
                    dict2[re_a] =1
                    #print(re_a_count)
                re_a_count+=1
                #print(i,re_a)
                #print(save_back)
                #print("_______")
        else:

            save_back=save_back+re_a+" "

            #print(save_back)
            #print("+========+")
            #print(re_a)

            if re_a in dict2:

                dict2[re_a]+=1

            else:
                dict2[re_a] =1
            #print(re_a_count)
            re_a_count+=1


    #save_back=list(save_back)
    save_back_list=[save_back]
    list_col.append(save_back_list)
    save_back=''
    save_back_list=[]

#print(dict2)
new_list1=sorted(dict2.items(),key=lambda item:item[1])
#print(new_list)
get_dict1=dict(new_list1)
print(get_dict1)
print(len(get_dict1))
#print(list_col)



In [ ]:
print(get_dict1)
print(len(get_dict1))

In [ ]:



import pandas as ps
data=ps.DataFrame(list_col,columns=['Ingredients_clean'])
print(data)
data.to_csv('database_back_jieba_cut_7.csv',header=True,index=False,encoding='utf-8-sig')

In [ ]:
test=(x['Ingredients'][0:51])
print(test)
test.to_csv("database_back_before2.csv",header=True,index=False,encoding='utf-8-sig')

In [ ]:
import jieba
import jieba.posseg as pseg
def jieba_cut(cut_word):
    #jieba.set_dictionary('./dict.txt')
    jieba.load_userdict('./user.txt')
    strs=cut_word
    for str1 in strs:
        seg_list = jieba.cut(str1,use_paddle=True) # 使用paddle模式
        print("Paddle Mode: " + ' '.join(list(seg_list)))

    
    finsh=[]
    text = cut_word
    print(text)
    words = pseg.cut(text)
    #jieba.enable_paddle()
    #words = pseg.cut(text,use_paddle=True)
    for word, flag in words:
        if flag =="n" or flag== "nr" or flag=='x':
            print(word, flag)
            finsh.append(word)
        else:
            print(word, flag)
            #pass
    print(finsh)
    return finsh
jieba_cut(["麵疙瘩材料~中筋麵粉"])          

In [ ]:
def jieba_cut2(cut_word):
    #jieba.set_dictionary('./dict.txt')
    jieba.load_userdict('./user.txt')
    cut_list=cut_word
    finsh=''
    for str1 in cut_list:
        seg_list = jieba.lcut(str1, cut_all=False)
        print(seg_list)
        a=(' '.join(seg_list))
        print(a)
        print(finsh)
        finsh=finsh+a
        print(finsh)

    #print(finsh)
    return finsh
    
jieba_cut2(["麵疙瘩材料~中筋麵粉"]) 
    #words = pseg.cut(text)
    #jieba.enable_paddle()
    #words = pseg.cut(text,use_paddle=True)
    #for word, flag in words:
        #if flag =="n" or flag== "nr" or flag=='x':
            #print(word, flag)
            #finsh.append(word)
        #else:
            #print(word, flag)
            #pass
    

In [ ]:
import jieba
jieba.set_dictionary('./dict.txt')
jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持
strs=["我来到北京清华大学","乒乓球拍卖完了","中国科学技术大学"]
for str in strs:
    seg_list = jieba.cut(str,use_paddle=True) # 使用paddle模式
    print("Paddle Mode: " + '/'.join(list(seg_list)))

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

In [ ]:
import jieba
import jieba.posseg as pseg
words = pseg.cut("我愛北京天安門") #jieba默认模式
jieba.enable_paddle() #启动paddle模式。 0.40版之后开始支持，早期版本不支持
words = pseg.cut("我愛北京天安門",use_paddle=True) #paddle模式
for word, flag in words:
    print('%s %s' % (word, flag))

In [ ]:
from __future__ import print_function, unicode_literals
import sys
sys.path.append("../")
import jieba
jieba.load_userdict("dict.txt")
import jieba.posseg as pseg

jieba.add_word('石墨烯')
jieba.add_word('凱特琳')
jieba.del_word('自定义词')

test_sent = (
"李小福是创新办主任也是云计算方面的专家; 什么是八一双鹿\n"
"例如我输入一个带“韩玉赏鉴”的标题，在自定义词库中也增加了此词为N类\n"
"「台中」正確應該不會被切開。mac上可分出「石墨烯」；此時又可以分出來凱特琳了。"
)
words = jieba.cut(test_sent)
print('/'.join(words))

print("="*40)

result = pseg.cut(test_sent)

for w in result:
    print(w.word, "/", w.flag, ", ", end=' ')

print("\n" + "="*40)

terms = jieba.cut('easy_install is great')
print('/'.join(terms))
terms = jieba.cut('python 的正则表达式是好用的')
print('/'.join(terms))

print("="*40)
# test frequency tune
testlist = [
('今天天气不错', ('今天', '天气')),
('如果放到post中将出错。', ('中', '将')),
('我们中出了一个叛徒', ('中', '出')),
]

for sent, seg in testlist:
    print('/'.join(jieba.cut(sent, HMM=False)))
    word = ''.join(seg)
    print('%s Before: %s, After: %s' % (word, jieba.get_FREQ(word), jieba.suggest_freq(seg, True)))
    print('/'.join(jieba.cut(sent, HMM=False)))
    print("-"*40)

In [ ]:
import pandas as pd

rowindex=['食材名稱']
df=pd.DataFrame(get_dict1,index=rowindex)
df=df.stack()
df.to_csv('database_clean4.csv',header=False,index=True,encoding='utf-8-sig')
y=pd.read_csv('database_clean4.csv',encoding='utf-8')
#print(y)
z=y.drop(columns='食材名稱',axis=1)
print(y.columns)
print(z)
z.to_csv('database_jieba_total_clean3.csv',header=True,index=False,encoding='utf-8-sig')